In [1]:
# | default_exp agent.planner.breakdown

In [2]:
# | export
from typing import Optional
from rich.console import Console
from agentic.agent.planner.models import ProjectBreakdown

In [3]:
# | export
class ProjectBreakdownGenerator:
    """Handles project breakdown generation and parsing"""
    
    def __init__(self, agent, console: Console):
        self.agent = agent
        self.console = console
    
    def generate_project_breakdown(self, user_request: str) -> Optional[ProjectBreakdown]:
        """Phase 1: Generate structured project breakdown with task-wise organization"""
        
        prompt = f"""
Analyze this project request and provide a structured breakdown for AI agent execution:

REQUEST: {user_request}

Provide response in this EXACT format:

PROJECT TITLE:
[Short, meaningful project name (2-4 words max, like "ScrapeMaster", "DataHarvester", "WebCrawler Pro")]

PROJECT SUMMARY:
[One paragraph overview of what needs to be built]

MAJOR PHASES:
Phase 1: [Phase name] - [Brief description]
Phase 2: [Phase name] - [Brief description]
Phase 3: [Phase name] - [Brief description]
[Continue as needed]

PHASE TASKS:
Phase 1 Tasks:
- Task 1.1: Analyze requirements and create specifications
- Task 1.2: Setup project structure and configuration
- Task 1.3: Initialize development environment

Phase 2 Tasks:
- Task 2.1: Implement core functionality
- Task 2.2: Create data models and interfaces
- Task 2.3: Build processing logic

[Continue for all phases with AI-executable tasks]

KEY DELIVERABLES:
- [Specific file/component 1]
- [Specific file/component 2]
- [Continue list]

TECHNICAL APPROACH:
[Architecture and technology decisions in 2-3 sentences]

Use AI automation terminology: analyze, generate, implement, validate, deploy.
Focus on tasks that can be executed programmatically by AI agent.
Avoid human-centric activities like interviews, meetings, or manual processes.
"""
        
        try:
            agent_response = self.agent.run(prompt)
            self.console.print(f"🔍 Agent response type: {type(agent_response)}")
            self.console.print(f"🔍 Agent response keys: {agent_response.keys() if isinstance(agent_response, dict) else 'Not a dict'}")
            
            response = agent_response.get("content", "") if isinstance(agent_response, dict) else str(agent_response)
            self.console.print(f"🔍 Response length: {len(response)}")
            
            # Save breakdown for reference
            # with open("project_breakdown.txt", 'w') as f:
            #     f.write(response)
            
            # Parse the response into structured format
            breakdown = self._parse_breakdown_response(response)
            
            # Save breakdown with title-based filename
            if breakdown and breakdown.title:
                filename = f"{breakdown.title.lower().replace(' ', '_')}_breakdown.txt"
                with open(filename, 'w') as f:
                    f.write(response)
            
            return breakdown
            
        except Exception as e:
            self.console.print(f"❌ Project breakdown generation failed: {e}")
            return None
    
    def _parse_breakdown_response(self, response: str) -> ProjectBreakdown:
        """Parse breakdown response into structured format"""
        
        lines = response.split('\n')
        
        title = "Untitled Project"
        project_summary = "Project breakdown generated"
        phases = []
        deliverables = []
        technical_approach = "Standard approach"
        
        current_section = None
        
        for line in lines:
            line = line.strip()
            if not line:
                continue
                
            # Detect sections
            if "PROJECT TITLE" in line.upper() or line.upper().startswith("TITLE"):
                current_section = "title"
            elif "PROJECT SUMMARY" in line.upper() or "SUMMARY" in line.upper():
                current_section = "summary"
            elif "PHASES" in line.upper() or "PHASE" in line.upper():
                current_section = "phases"
            elif "DELIVERABLES" in line.upper():
                current_section = "deliverables"
            elif "TECHNICAL" in line.upper():
                current_section = "technical"
            elif line.startswith('-') or line.startswith('•') or line.startswith('*'):
                # List item
                item = line.lstrip('-•* ').strip()
                if current_section == "phases":
                    phases.append(item)
                elif current_section == "deliverables":
                    deliverables.append(item)
            elif current_section == "title" and len(line) > 3:
                title = line
            elif current_section == "summary" and len(line) > 20:
                project_summary = line
            elif current_section == "technical" and len(line) > 20:
                technical_approach = line
        
        return ProjectBreakdown(
            title=title,
            project_summary=project_summary,
            phases=phases if phases else ["Setup", "Development", "Testing", "Deployment"],
            key_deliverables=deliverables if deliverables else ["Core components", "Documentation", "Tests"],
            technical_approach=technical_approach
        )
    
    def estimate_total_tasks(self, breakdown: ProjectBreakdown) -> int:
        """Estimate total number of tasks based on project breakdown"""
        task_count = 0
        breakdown_text = "\n".join(breakdown.phases)
        for line in breakdown_text.split('\n'):
            if line.strip().startswith('-') or line.strip().startswith('•') or line.strip().startswith('*'):
                task_count += 1
        return max(task_count, 5)

